## Initialization

In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.datasets import fetch_openml
from sklearn.preprocessing import LabelEncoder, OrdinalEncoder, StandardScaler
from sklearn.model_selection import train_test_split
import random
import os
import sys

In [2]:
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.append(project_root)

In [3]:
output_dir = os.path.join(project_root, 'outputs')
os.makedirs(output_dir, exist_ok=True)

In [4]:
from fairness.losses import fair_bce_loss, calculate_alpha
from fairness.metrics import accuracy_equality, statistical_parity, equal_opportunity, predictive_equality

In [5]:
def set_seed(seed=123):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

In [6]:
set_seed(123)

In [7]:
# Selecting GPU if configured
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

Using device: cuda


## Load & Transform Dataset

In [8]:
dataset = fetch_openml("adult", version=2, as_frame=True)
df = dataset.data.copy()

In [9]:
df['target'] = dataset.target
df['sex'] = LabelEncoder().fit_transform(df['sex'])

In [10]:
# Handling NaN values
if "Unknown" not in df['native-country'].cat.categories:
    df['native-country'] = df['native-country'].cat.add_categories(["Unknown"])
df['native-country'] = df['native-country'].fillna("Unknown")

if "Unemployed" not in df['workclass'].cat.categories:
    df['workclass'] = df['workclass'].cat.add_categories(["Unemployed"])
df['workclass'] = df['workclass'].fillna("Unemployed")

if "Unemployed" not in df['occupation'].cat.categories:
    df['occupation'] = df['occupation'].cat.add_categories(["Unemployed"])
df['occupation'] = df['occupation'].fillna("Unemployed")

In [11]:
# Features selection and Transformation
selected_cols = ['age', 'workclass', 'education-num', 'marital-status', 'occupation', 'relationship',
                 'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week', 'native-country']
X = OrdinalEncoder().fit_transform(df[selected_cols])
X = pd.DataFrame(X, columns=selected_cols)

In [12]:
# Target selection and Transformation
y = LabelEncoder().fit_transform(df['target'])
y = pd.DataFrame(y, columns=['target'])

In [13]:
# Train, validation, test split
X_train, X_valtest, y_train, y_valtest, sex_train, sex_valtest = train_test_split(
    X, y, df['sex'], test_size=0.3, random_state=42, stratify=df['sex']
)

X_val, X_test, y_val, y_test, sex_val, sex_test = train_test_split(
    X_valtest, y_valtest, sex_valtest, test_size = 0.5, random_state=42, stratify=sex_valtest
)

In [14]:
# Standarization
scaler = StandardScaler()

X_train = torch.tensor(scaler.fit_transform(X_train), dtype=torch.float32)
X_val = torch.tensor(scaler.transform(X_val), dtype=torch.float32)
X_test = torch.tensor(scaler.transform(X_test), dtype=torch.float32)

y_train = torch.tensor(y_train.values, dtype=torch.float32)
y_val = torch.tensor(y_val.values, dtype=torch.float32)
y_test = torch.tensor(y_test.values, dtype=torch.float32)

sex_train = torch.tensor(sex_train.values, dtype=torch.float32)
sex_val = torch.tensor(sex_val.values, dtype=torch.float32)
sex_test = torch.tensor(sex_test.values, dtype=torch.float32)

## Creating ML Models

In [15]:
# MLP Model
class MLP(nn.Module):
    def __init__(self, input_dim):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_dim, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 16)
        self.fc4 = nn.Linear(16, 8)
        self.fc5 = nn.Linear(8, 1)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()
       
    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.relu(self.fc3(x))
        x = self.relu(self.fc4(x))
        x = self.sigmoid(self.fc5(x))
        return x  

In [16]:
# Logistic Regression Model
class LogisticRegression(nn.Module):
    def __init__(self, input_dim) :
        super(LogisticRegression, self).__init__()
        self.linear = nn.Linear(input_dim, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x): 
        return self.sigmoid(self.linear(x))

In [17]:
# Loading ML Model
def load_ml_model(ml_algorithm="None"):
    if ml_algorithm == "LR":
        return LogisticRegression(input_dim=X_train.shape[1])
    elif ml_algorithm == "MLP":
        return MLP(input_dim=X_train.shape[1])
    else: 
        raise ValueError(f"Unsupported ML algorithm: {ml_algorithm}")

## Training & Validating Models

In [18]:
def train_model(model, optimizer, epochs, patience, alpha, alpha_mode, fairness_mode):
    best_val_acc = 0.0
    best_model_state = None
    epochs_no_improve = 0

    for epoch in range(epochs):
        model.train()
        optimizer.zero_grad()

        outputs = model(X_train)
        current_alpha = calculate_alpha(epoch, epochs, alpha, alpha_mode)
        loss = fair_bce_loss(outputs, y_train, sex_train, alpha=current_alpha, fairness_mode=fairness_mode)

        loss.backward()
        optimizer.step()

        model.eval()
        with torch.no_grad():
            val_outputs = model(X_val)
            acc_val = ((val_outputs > 0.5).float() == y_val).float().mean().item()

        if acc_val > best_val_acc:
            best_val_acc = acc_val
            best_model_state = model.state_dict()
            epochs_no_improve = 0
        else:
            epochs_no_improve += 1

        if epochs_no_improve > patience:
            break

    return best_model_state

## Testing Models

In [19]:
def test_model(model, fairness_mode):
    model.eval()
    with torch.no_grad():
        test_outputs = model(X_test)
        test_preds = (test_outputs > 0.5).float()
        test_acc = (test_preds == y_test).float().mean().item()

        if fairness_mode == "AE":
            fairness_score, _ = accuracy_equality(test_outputs.squeeze(), y_test, sex_test)
        elif fairness_mode == "SP":
            fairness_score, _ = statistical_parity(test_outputs, sex_test)
        elif fairness_mode == "EOP":
            fairness_score, _ = equal_opportunity(test_outputs, y_test.squeeze(), sex_test)
        elif fairness_mode == "PE":
            fairness_score, _ = predictive_equality(test_outputs, y_test.squeeze(), sex_test)
        else:
            raise ValueError(f"Unsupported fairness mode: {fairness_mode}")

    return test_acc, fairness_score

## Main functions

In [20]:
def save_results(results, output_dir, model_score, model_acc, model_fair):
    # Save models
    torch.save(model_score, os.path.join(output_dir, "best_model_score.pth"))
    torch.save(model_acc, os.path.join(output_dir, "best_model_acc.pth"))
    torch.save(model_fair, os.path.join(output_dir, "best_model_fair.pth"))

    # Save CSV
    df_results = pd.DataFrame(results)
    results_file = os.path.join(output_dir, "results.csv")
    file_exists = os.path.exists(results_file)
    df_results.to_csv(results_file, mode='a', header=not file_exists, index=False)
    
    return df_results

In [21]:
def run_all_experiments(ml_algorithm, epochs, alpha_values, alpha_modes, fairness_mode, patience=100):
    results = []

    best_score, best_acc, best_fairness = -float("inf"), -float("inf"), float("inf")
    best_model_score, best_model_acc, best_model_fair = None, None, None
    best_result_score, best_result_acc, best_result_fair = None, None, None

    for alpha in alpha_values:
        for alpha_mode in alpha_modes:
            print(f"\nRunning {ml_algorithm} | alpha={alpha}, mode={alpha_mode}, fairness={fairness_mode}")

            model = load_ml_model(ml_algorithm)
            optimizer = optim.Adam(model.parameters(), lr=0.01)

            best_model_state = train_model(model, optimizer, epochs, patience, alpha, alpha_mode, fairness_mode)
            model.load_state_dict(best_model_state)

            test_acc, fairness_score = test_model(model, fairness_mode)

            print(f"Test Accuracy: {test_acc:.3f}, {fairness_mode}: {fairness_score:.3f}")

            result_row = {
                "ml_algorithm": ml_algorithm,
                "fairness_mode": fairness_mode,
                "fairness_score": fairness_score,
                "test_accuracy": test_acc,
                "alpha_mode": alpha_mode,
                "alpha_value": alpha
            }

            results.append(result_row)

            score = test_acc - fairness_score

            # Best score
            if score > best_score:
                best_score = score
                best_model_score = best_model_state
                best_result_score = result_row

            # Best accuracy
            if test_acc > best_acc or (test_acc == best_acc and fairness_score < best_result_acc["fairness_score"]):
                best_acc = test_acc
                best_model_acc = best_model_state
                best_result_acc = result_row

            # Best fairness (lowest value)
            if fairness_score < best_fairness or (fairness_score == best_fairness and test_acc > best_result_fair["test_accuracy"]):
                best_fairness = fairness_score
                best_model_fair = best_model_state
                best_result_fair = result_row

    # Report best results
    print("\nBest score config:", best_result_score)
    print("\nBest accuracy config:", best_result_acc)
    print("\nBest fairness config:", best_result_fair)

    df_results = save_results(results, output_dir, best_model_score, best_model_acc, best_model_fair)
    
    return df_results

## User Input + Run

In [22]:
df = run_all_experiments(
    ml_algorithm="MLP",  # Models: MLP, LR
    epochs=100,
    alpha_values=[0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0],  # Alpha range between 0 - 1
    alpha_modes=["const", "linear_increase", "linear_decrease"],  # Alpha modes: const, linear_decrease, linear_increase
    fairness_mode="SP"  # Fairness modes: AE, SP, EOP, PE
)


Running MLP | alpha=0.0, mode=const, fairness=SP
Test Accuracy: 0.246, SP: 0.000

Running MLP | alpha=0.0, mode=linear_increase, fairness=SP
Test Accuracy: 0.754, SP: 0.000

Running MLP | alpha=0.0, mode=linear_decrease, fairness=SP
Test Accuracy: 0.843, SP: 0.174

Running MLP | alpha=0.1, mode=const, fairness=SP
Test Accuracy: 0.754, SP: 0.000

Running MLP | alpha=0.1, mode=linear_increase, fairness=SP
Test Accuracy: 0.809, SP: 0.006

Running MLP | alpha=0.1, mode=linear_decrease, fairness=SP
Test Accuracy: 0.843, SP: 0.121

Running MLP | alpha=0.2, mode=const, fairness=SP
Test Accuracy: 0.805, SP: 0.003

Running MLP | alpha=0.2, mode=linear_increase, fairness=SP
Test Accuracy: 0.823, SP: 0.022

Running MLP | alpha=0.2, mode=linear_decrease, fairness=SP
Test Accuracy: 0.839, SP: 0.060

Running MLP | alpha=0.3, mode=const, fairness=SP
Test Accuracy: 0.813, SP: 0.014

Running MLP | alpha=0.3, mode=linear_increase, fairness=SP
Test Accuracy: 0.832, SP: 0.037

Running MLP | alpha=0.3, mo